In [22]:
import torch
from PIL import Image
import pathlib
import numpy as np
import cv2
import matplotlib.pyplot as plt 
import time

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 2.2.0.dev20231001 CPU


In [23]:
from ultralytics import YOLO

In [24]:
model = YOLO('/Users/ameyakore/Downloads/last-6.pt')
model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [ ]:
model.train(data="/Users/ameyakore/workspaces/ai-workspace/dvl/notebooks/data/car_bus_truck_Detection/data.yaml", epochs=14, cache="ram", batch=64) 




In [ ]:
img1p = pathlib.Path("data/car_bus_truck_Detection_2/test/images/")

In [ ]:
for it in img1p.iterdir():
    out = model.predict(it, save_crop=True)

In [ ]:
model.predict(source="data/car_bus_truck_Detection_2/test/images/")

In [6]:
classes = {0: 'car', 1: 'large_bus', 2: 'medium_truck'}
# classes = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat", "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed", "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"]
COLORS = np.array([[255.0, 0, 0], [0, 255, 0], [0, 0, 255]])
# COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

def draw_bounding_box(img, class_id, confidence, x, y, x2, y2):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x2,y2), color, 2)

    cv2.putText(img, label + ": "+ "{:.2f}".format(confidence), (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)


def draw_bounding_image(img, mask, class_id, confidence, x, y, x2, y2):
    label = str(classes[class_id])

    color = COLORS[class_id]

    mask[y:y2,x:x2] = img[y:y2,x:x2]

    cv2.rectangle(mask, (x,y), (x2,y2), color, 2)

    cv2.putText(mask, label + ": "+ "{:.2f}".format(confidence), (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)



In [7]:
tstimg = pathlib.Path("/Users/ameyakore/Downloads/images-2.jpeg")
out = model.predict(tstimg, show=True)
print(out[0].boxes)
cv2.waitKey(0)

2023-10-11 12:37:28.530 Python[67928:12013149] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.

image 1/1 /Users/ameyakore/Downloads/images-2.jpeg: 640x640 1 large_bus, 75.8ms
Speed: 1.2ms preprocess, 75.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1.])
conf: tensor([0.3212])
data: tensor([[1.3838e+02, 1.7371e+02, 2.7046e+02, 3.6787e+02, 3.2118e-01, 1.0000e+00]])
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([1, 6])
xywh: tensor([[204.4205, 270.7924, 132.0867, 194.1578]])
xywhn: tensor([[0.3194, 0.4231, 0.2064, 0.3034]])
xyxy: tensor([[138.3771, 173.7135, 270.4639, 367.8713]])
xyxyn: tensor([[0.2162, 0.2714, 0.4226, 0.5748]])


113

In [8]:
def predict_and_draw_on_frame(frame, imgsz=(640, 640)):
    out1 = model(frame, conf=0.01, classes=[0,2], imgsz=imgsz)
    out2 = model(frame, conf=0.75, classes=[1], imgsz=imgsz)
    # out3 = model(frame, conf=0.1, classes=[2])
    for ou in out1:
        for c, co, xywh in zip(ou.boxes.cls, ou.boxes.conf, ou.boxes.xyxy):
                draw_bounding_box(frame, int(c.item()), co.item(), int(xywh[0].item()), int(xywh[1].item()),int(xywh[2].item()), int(xywh[3].item()))
    
    for ou in out2:
        for c, co, xywh in zip(ou.boxes.cls, ou.boxes.conf, ou.boxes.xyxy):
                draw_bounding_box(frame, int(c.item()), co.item(), int(xywh[0].item()), int(xywh[1].item()),int(xywh[2].item()), int(xywh[3].item()))
    
    
    # for ou in out3:
    #     for c, co, xywh in zip(ou.boxes.cls, ou.boxes.conf, ou.boxes.xyxy):
    #             draw_bounding_box(frame, int(c.item()), co.item(), int(xywh[0].item()), int(xywh[1].item()),int(xywh[2].item()), int(xywh[3].item()))

    return frame

def predict_normal(frame):
    out1 = model(frame)
    for ou in out1:
        print(ou.boxes)
        for c, co, xywh in zip(ou.boxes.cls, ou.boxes.conf, ou.boxes.xyxy):
                draw_bounding_box(frame, int(c.item()), co.item(), int(xywh[0].item()), int(xywh[1].item()),int(xywh[2].item()), int(xywh[3].item()))
    
    return frame


def predict_and_draw_on_mask(frame, mask):
    out1 = model(frame, conf=0.01, classes=[0,2])
    out2 = model(frame, conf=0.75, classes=[1])
    # out3 = model(frame, conf=0.1, classes=[2])
    for ou in out1:
        for c, co, xywh in zip(ou.boxes.cls, ou.boxes.conf, ou.boxes.xyxy):
                draw_bounding_image(frame, mask, int(c.item()), co.item(), int(xywh[0].item()), int(xywh[1].item()),int(xywh[2].item()), int(xywh[3].item()))
    
    for ou in out2:
        for c, co, xywh in zip(ou.boxes.cls, ou.boxes.conf, ou.boxes.xyxy):
                draw_bounding_image(frame, mask, int(c.item()), co.item(), int(xywh[0].item()), int(xywh[1].item()),int(xywh[2].item()), int(xywh[3].item()))
    return mask
    


In [9]:
frame = cv2.imread("/Users/ameyakore/Downloads/images-2.jpeg")

frame = predict_and_draw_on_frame(frame)

cv2.imwrite("tst.jpeg", frame)
    


0: 640x640 12 cars, 5 medium_trucks, 77.1ms
Speed: 2.2ms preprocess, 77.1ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 67.0ms
Speed: 1.1ms preprocess, 67.0ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 640)


True

In [10]:
frame = cv2.imread("/Users/ameyakore/Downloads/originaltest.jpg")

frame = predict_and_draw_on_frame(frame)

cv2.imwrite("tst.jpeg", frame)
    


0: 640x640 19 cars, 6 medium_trucks, 72.1ms
Speed: 2.0ms preprocess, 72.1ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 68.2ms
Speed: 1.3ms preprocess, 68.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)


True

In [11]:
for it in pathlib.Path("data/car_bus_truck_Detection_2/test/images").iterdir():
    frame = cv2.imread(str(it))

    frame = predict_normal(frame)

    cv2.imshow("tst.jpeg", frame)

    cv2.waitKey(2000) & 0xFF == ord('q')
    


0: 640x640 4 large_buss, 72.0ms
Speed: 1.7ms preprocess, 72.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 large_bus, 68.0ms
Speed: 1.3ms preprocess, 68.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1., 1., 1., 1.])
conf: tensor([0.8990, 0.8861, 0.8646, 0.8281])
data: tensor([[173.4313, 159.5974, 363.0793, 612.4052,   0.8990,   1.0000],
        [253.3825,   0.0000, 429.2690, 204.0071,   0.8861,   1.0000],
        [430.4995,   0.0000, 551.3918, 189.9637,   0.8646,   1.0000],
        [552.3604,   0.0000, 639.7989, 154.3849,   0.8281,   1.0000]])
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([4, 6])
xywh: tensor([[268.2553, 386.0013, 189.6479, 452.8078],
        [341.3257, 102.0036, 175.8865, 204.0071],
        [490.9456,  94.9818, 120.8924, 189.9637],
        [596.0797,  77.1925,  87.4385, 154.3849]])
xywhn: tensor([[0.4191, 0.6031, 0.2963, 0.7075],
        [0.5333, 0.1594, 0.2748, 0.3188],
        [0.7671, 0.1484, 0.1889, 0.2968],
        [0.9314, 0.1206, 0.1366, 0.2412]])
xyxy: tensor([[173.4313, 159.5974, 363.0793, 612.4052],
        [253.3825,   0.0000, 429.2690, 204.0071],
        [430.49


0: 640x640 3 large_buss, 87.8ms
Speed: 3.1ms preprocess, 87.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 large_buss, 65.5ms
Speed: 1.1ms preprocess, 65.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1., 1., 1.])
conf: tensor([0.8874, 0.6200, 0.6159])
data: tensor([[2.2721e+02, 4.3203e+02, 4.0893e+02, 5.6918e+02, 8.8739e-01, 1.0000e+00],
        [1.8278e+01, 1.3863e-02, 5.3664e+01, 1.9548e+01, 6.2000e-01, 1.0000e+00],
        [1.3053e-01, 1.7836e-01, 1.6837e+01, 3.6547e+01, 6.1595e-01, 1.0000e+00]])
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([3, 6])
xywh: tensor([[318.0686, 500.6044, 181.7160, 137.1572],
        [ 35.9708,   9.7807,  35.3865,  19.5337],
        [  8.4838,  18.3627,  16.7066,  36.3686]])
xywhn: tensor([[0.4970, 0.7822, 0.2839, 0.2143],
        [0.0562, 0.0153, 0.0553, 0.0305],
        [0.0133, 0.0287, 0.0261, 0.0568]])
xyxy: tensor([[2.2721e+02, 4.3203e+02, 4.0893e+02, 5.6918e+02],
        [1.8278e+01, 1.3863e-02, 5.3664e+01, 1.9548e+01],
        [1.3053e-01, 1.7836e-01, 1.6837e+01, 3.6547e+01]])
xyxyn: tensor([[3.5502e-01, 6.7504e-01, 6.3895e-01, 8.8935e-01],
        [2.85


0: 640x640 2 large_buss, 67.4ms
Speed: 1.6ms preprocess, 67.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 large_buss, 66.5ms
Speed: 1.6ms preprocess, 66.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)



...

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1.])
conf: tensor([0.8846])
data: tensor([[215.1115, 224.7993, 514.2195, 491.8813,   0.8846,   1.0000]])
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([1, 6])
xywh: tensor([[364.6655, 358.3403, 299.1080, 267.0820]])
xywhn: tensor([[0.5698, 0.5599, 0.4674, 0.4173]])
xyxy: tensor([[215.1115, 224.7993, 514.2195, 491.8813]])
xyxyn: tensor([[0.3361, 0.3512, 0.8035, 0.7686]])
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1., 1.])
conf: tensor([0.9033, 0.8925])
data: tensor([[481.6828,  67.9766, 638.8679, 538.1474,   0.9033,   1.0000],
        [161.0806, 133.8330, 435.2845, 585.1836,   0.8925,   1.0000]])
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([2, 6])
xywh: tensor([[560.2753, 303.0620, 157.1851, 470.1708],
        [298.1826, 359.5083, 274.2039, 451.3506]])
xywhn: tensor([[0.8754, 0.4735, 0.2456, 0.7346],
        [0.4659, 0.5617, 0.4284, 0.7052]])
xyxy:

0: 640x640 1 large_bus, 67.4ms
Speed: 1.1ms preprocess, 67.4ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 large_buss, 64.4ms
Speed: 1.0ms preprocess, 64.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)



ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1.])
conf: tensor([0.9018])
data: tensor([[155.4787, 277.2880, 364.4680, 521.3195,   0.9018,   1.0000]])
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([1, 6])
xywh: tensor([[259.9733, 399.3038, 208.9893, 244.0315]])
xywhn: tensor([[0.4062, 0.6239, 0.3265, 0.3813]])
xyxy: tensor([[155.4787, 277.2880, 364.4680, 521.3195]])
xyxyn: tensor([[0.2429, 0.4333, 0.5695, 0.8146]])
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1., 1.])
conf: tensor([0.8782, 0.8722])
data: tensor([[ 44.9645, 298.0424, 177.0363, 419.4987,   0.8782,   1.0000],
        [  0.0000, 172.9402,  50.3080, 272.4214,   0.8722,   1.0000]])
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([2, 6])
xywh: tensor([[111.0004, 358.7706, 132.0718, 121.4563],
        [ 25.1540, 222.6808,  50.3080,  99.4811]])
xywhn: tensor([[0.1734, 0.5606, 0.2064, 0.1898],
        [0.0393, 0.3479, 0.0786, 0.1554]])
xyxy:

0: 640x640 4 large_buss, 67.2ms
Speed: 1.2ms preprocess, 67.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)


ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1., 1., 1., 1.])
conf: tensor([0.9048, 0.8999, 0.6763, 0.3418])
data: tensor([[3.0072e+02, 2.7510e+02, 5.2275e+02, 4.2408e+02, 9.0477e-01, 1.0000e+00],
        [5.6836e+01, 2.7546e+02, 2.2725e+02, 4.5710e+02, 8.9990e-01, 1.0000e+00],
        [0.0000e+00, 3.6942e+02, 2.5767e+01, 4.8065e+02, 6.7629e-01, 1.0000e+00],
        [0.0000e+00, 3.1759e+02, 2.2143e+01, 4.7899e+02, 3.4181e-01, 1.0000e+00]])
id: None
is_track: False
orig_shape: (640, 640)
shape: torch.Size([4, 6])
xywh: tensor([[411.7355, 349.5886, 222.0375, 148.9762],
        [142.0429, 366.2791, 170.4131, 181.6418],
        [ 12.8834, 425.0360,  25.7667, 111.2263],
        [ 11.0714, 398.2881,  22.1428, 161.4040]])
xywhn: tensor([[0.6433, 0.5462, 0.3469, 0.2328],
        [0.2219, 0.5723, 0.2663, 0.2838],
        [0.0201, 0.6641, 0.0403, 0.1738],
        [0.0173, 0.6223, 0.0346, 0.2522]])
xyxy: tensor([[300.7168, 275.1005, 522.7543, 424.0767],
        [ 56.8363

In [12]:
def crop_poly(img, pts):
    
    mask = np.ones(img.shape[:2], np.uint8) * 255
    cv2.drawContours(mask, [pts], -1, (0, 0, 0), -1, cv2.LINE_AA)
    
    ## (3) do bit-op
    dst = cv2.bitwise_and(img, img, mask=mask)

    return dst

def crop_poly_white(img, pts):
    
    mask = np.ones(img.shape[:2], np.uint8) * 255
    cv2.drawContours(mask, [pts], -1, (0,0,0), -1, cv2.LINE_AA)
    
    ## (3) do bit-op
    dst = cv2.bitwise_and(img, img, mask=mask)
    
    ## (4) add the white background
    bg = np.ones_like(img, np.uint8)*255
    cv2.bitwise_not(bg,bg, mask=mask)
    dst2 = bg+ dst
    
    return dst2

In [13]:
cap = cv2.VideoCapture('/Users/ameyakore/Downloads/production_id_4979728 (1080p).mp4')
count = 0
crop_h = 0
scale_factor = 1
while cap.isOpened():
    ret,frame = cap.read()
    frame = cv2.resize(frame, (640 * scale_factor, 640 * scale_factor))
    frame = frame[crop_h * scale_factor:,:]

    #Segmentation starts

    #For top not necessary if top is cropped
    frame = crop_poly_white(frame, np.array([[0, 0], [0, 317], [640, 317], [640, 0]]))
    
    # For center 
    frame = crop_poly_white(frame, np.array([[277 * scale_factor, (380 - crop_h) * scale_factor],[410 * scale_factor, (392 - crop_h)* scale_factor],[381* scale_factor, (506 - crop_h)* scale_factor],[155* scale_factor, (498 - crop_h)* scale_factor]]))

    #For left side
    frame = crop_poly_white(frame, np.array([[97* scale_factor, 0],[97* scale_factor, (365 - crop_h)* scale_factor],[160* scale_factor, (390 - crop_h)* scale_factor],[0, (462 - crop_h)* scale_factor], [0, 0]]))

    #For right side
    frame = crop_poly_white(frame, np.array([[556* scale_factor, 0],[556* scale_factor, (365 - crop_h)* scale_factor],[504* scale_factor, (370 - crop_h)* scale_factor],[542* scale_factor, (444 - crop_h)* scale_factor], [640* scale_factor, (467 - crop_h)* scale_factor], [640* scale_factor, 0]]))
    
    # blank_image_mask = np.zeros((640,640,3), np.uint8)
    #frame = predict_and_draw_on_frame(frame, ((640 - crop_h)* scale_factor, 640* scale_factor))
    # frame = predict_and_draw_on_mask(frame, blank_image_mask)
    cv2.imshow('video', frame)
    #cv2.imwrite("frame%d.jpg" % count, frame)
    count = count + 1
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
    

cap.release()
cv2.destroyAllWindows() # destroy all opened windows

In [14]:
def time_profile(path, crop_h, scale_factor=1):
    timings = []
    cap = cv2.VideoCapture(path)
    count = 0
    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            break
        t1 = time.time()
        model.predict(frame, imgsz = ((640 - crop_h)* scale_factor, 640* scale_factor))
        # frame = predict_and_draw_on_frame(frame, ((640 - crop_h)* scale_factor, 640* scale_factor))
        t2 = time.time()
        timings.append(t2 - t1)
        # cv2.imshow('video', frame)
        # cv2.imwrite("frame%d.jpg" % count, frame)
        count = count + 1
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
        
    
    cap.release()
    cv2.destroyAllWindows() # destroy all opened windows

    return timings

In [ ]:
timings = time_profile('moviecrop.mp4', 317)

In [ ]:
timings2 = time_profile('movienocrop.mp4', 0)

In [ ]:
timings3 = time_profile('movienocroponlywhite.mp4', 0)

In [ ]:
import pandas as pd
plt.ylim(0, 0.15)
plt.plot(timings, label="crop and whited out")
plt.plot(timings2, label="no processing")
plt.plot(timings3, label="whited out but no crop")
plt.legend()
print(np.average(timings))
print(np.average(timings2))
print(np.average(timings3))
df = pd.DataFrame()
df['crop and whited out'] = timings
df['no processing'] = timings2
df['whited out but no crop'] = timings3
df.describe()

In [ ]:
model.predict('/Users/ameyakore/Downloads/production_id_4979728 (1080p).mp4', show=True)

In [25]:
import math
# Run Algo for processing
def calculate_pre(rois):
    # find bounding rect of all rois
    rects = [cv2.boundingRect(r) for r in rois]
    dims = math.ceil(math.sqrt(len(rects)))

    # Now find the mappings
    # return a list of source_x, source_y, dest_x, dest_y, w, h
    ans = []
    x_cur = 0
    y_cur = 0

    w_max = 0
    h_max = 0
    adsroi = []
    for idx, (rect, roi) in enumerate(zip(rects, rois)):
        # find rows to remove from image
        idx += 1
        x, y, w, h = rect

        off_x = x - x_cur
        off_y = y  - y_cur
        
        ans.append([x, y, x_cur, y_cur, w, h])

        adsroi.append(np.array([(x - off_x, y - off_y) for x, y, in roi]))
        
        x_cur += w

        if w_max < x_cur:
            w_max = x_cur

        if h_max < y_cur + h:
            h_max = y_cur + h
        
        if (dims != 1 and idx % dims == 0) or (dims == 1 and idx == dims):
            y_cur += h
            x_cur = 0
        
    return ans, (w_max, h_max), adsroi

def pre_process(image, ans, w, h):
    new_img = np.ones((h,w,3), np.uint8) * 255
    
    return new_img

def draw_contours(new_img, rois):
    new_img_2 = np.ones(new_img.shape, np.uint8) * 255
    mask_0 = np.ones(new_img.shape[:-1], np.uint8)
    for roi in rois:
        mask = np.ones(new_img.shape[:-1], np.uint8)
        cv2.drawContours(mask, [roi], -1, 0, -1, cv2.LINE_AA)
        cv2.drawContours(new_img_2, [roi], -1, (0, 0, 0), -1, cv2.LINE_AA)
        mask_0 += mask

       
    new_img_2 = cv2.bitwise_or(new_img, new_img_2, mask=mask_0)
    return new_img_2


def pre_processroi(image, rois, ans, w, h):
    new_img = np.ones((h,w,3), np.uint8) * 255
    for source_x, source_y, dest_x, dest_y, w, h in ans:
        new_img[dest_y: dest_y + h, dest_x: dest_x + w] = image[source_y:source_y + h, source_x : source_x+ w]
    
    return draw_contours(new_img, rois)

In [26]:
def time_profile_other_no_crop(path, ans, rois, imgsz = (640, 640)):
    timings = []
    processings = []
    print(imgsz)
    cap = cv2.VideoCapture(path)
    count = 0
    old_frame = None
    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            break
        t1 = time.time()
        #frame = pre_process(frame, ans, imgsz[0], imgsz[1])
        frame = draw_contours(frame, rois)
        #frame = pre_processroi(frame, rois, ans, imgsz[0], imgsz[1])
        #print(frame.shape[:2])
        t2 = time.time()
        timings.append(t2 - t1)
        old_frame = frame
        t1 = time.time()
        model.predict(frame, imgsz=imgsz, show=False)
        # frame = predict_and_draw_on_frame(frame, ((640 - crop_h)* scale_factor, 640* scale_factor))
        t2 = time.time()
        processings.append(t2 - t1)
        #cv2.imshow('video', frame)
        # cv2.imwrite("frame%d.jpg" % count, frame)
        count = count + 1
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cv2.imwrite("framenocrop.jpg", old_frame)        
    
    cap.release()
    cv2.destroyAllWindows() # destroy all opened windows

    return timings, processings

In [27]:
def time_profile_other_crop(path, ans, rois, imgsz = (640, 640)):
    timings = []
    processings = []
    print(imgsz)
    cap = cv2.VideoCapture(path)
    count = 0
    old_frame = None
    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            break
        t1 = time.time()
        #frame = pre_process(frame, ans, imgsz[0], imgsz[1])
        #frame = draw_contours(frame, rois)
        frame = pre_processroi(frame, rois, ans, imgsz[0], imgsz[1])
        #print(frame.shape[:2])
        t2 = time.time()
        timings.append(t2 - t1)
        old_frame = frame
        t1 = time.time()
        model.predict(frame, imgsz=imgsz, show=False)
        # frame = predict_and_draw_on_frame(frame, ((640 - crop_h)* scale_factor, 640* scale_factor))
        t2 = time.time()
        processings.append(t2 - t1)
        #cv2.imshow('video', frame)
        # cv2.imwrite("frame%d.jpg" % count, frame)
        count = count + 1
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cv2.imwrite("frameoccrop.jpg", old_frame)    
    
    cap.release()
    cv2.destroyAllWindows() # destroy all opened windows

    return timings, processings

In [28]:
pth = 'movienocrop.mp4'
# rois = [np.array([[0, 0], [0, 317], [640, 317], [640, 0]]),
  #              np.array([[277 * scale_factor, (380 - crop_h) * scale_factor],[410 * scale_factor, (392 - crop_h)* scale_factor],[381* scale_factor, (506 - crop_h)* scale_factor],[155* scale_factor, (498 - crop_h)* scale_factor]]),
   #             np.array([[97* scale_factor, 0],[97* scale_factor, (365 - crop_h)* scale_factor],[160* scale_factor, (390 - crop_h)* scale_factor],[0, (462 - crop_h)* scale_factor], [0, 0]]),
    #            np.array([[556* scale_factor, 0],[556* scale_factor, (365 - crop_h)* scale_factor],[504* scale_factor, (370 - crop_h)* scale_factor],[542* scale_factor, (444 - crop_h)* scale_factor], [640* scale_factor, (467 - crop_h)* scale_factor], [640* scale_factor, 0]])
     #            ]
rois = [np.array([[31, 452], [135, 391], [245, 398], [155, 484]]), np.array([[408, 400], [495, 416], [534, 523], [387, 503]]), 
        np.array([[425, 318], [499, 317], [528, 356], [418, 352]]), np.array([[184, 504], [346, 513], [384, 636], [37, 637]])]
cap = cv2.VideoCapture(pth)
ret,frame = cap.read()
ans, wh, rois_ads = calculate_pre(rois)

cap.release()
cv2.destroyAllWindows() # destroy all opened windows
                

In [29]:
timings, p1 = time_profile_other_no_crop('movienocrop.mp4', ans, rois)

(640, 640)


0: 640x640 (no detections), 84.8ms
Speed: 1.6ms preprocess, 84.8ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 large_bus, 73.4ms
Speed: 1.0ms preprocess, 73.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 large_bus, 76.0ms
Speed: 1.6ms preprocess, 76.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 large_bus, 74.6ms
Speed: 1.4ms preprocess, 74.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 large_bus, 74.7ms
Speed: 1.1ms preprocess, 74.7ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 large_bus, 73.6ms
Speed: 2.0ms preprocess, 73.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 large_bus, 68.7ms
Speed: 1.2ms preprocess, 68.7ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 large_bus, 75.0ms
Speed: 2.7ms preprocess, 75.0ms inference, 0.4ms postprocess p

KeyboardInterrupt: 

In [ ]:
timings_2, p2 = time_profile_other_crop('movienocrop.mp4', ans, rois_ads, wh)

In [ ]:
plt.plot(timings, label="crop and whited out")

In [ ]:
plt.plot(timings_2, label="no processing")

In [ ]:
import pandas as pd
plt.ylim(0, 0.003)
plt.plot(timings, label="segmented but not cropped")
plt.plot(timings_2, label="segmented and cropped")
plt.legend()
print(np.average(timings))
print(np.average(timings_2))
df = pd.DataFrame()
df['segmented but not cropped'] = timings
df['segmented and cropped'] = timings_2
df.describe()

In [ ]:
import pandas as pd
plt.plot(p1, label="processing segmented but not cropped")
plt.plot(p2, label="processing segmented and cropped")
plt.legend()
print(np.average(p1))
print(np.average(p2))
print(np.average(p1) / np.average(p2))
df = pd.DataFrame()
df['processing segmented but not cropped'] = p1
df['processing segmented and cropped'] = p2
df.describe()

In [ ]:
from PIL import Image
Image.open('framenocrop.jpg')

In [ ]:
from PIL import Image
Image.open('frameoccrop.jpg')

In [ ]:
import pandas as pd
plt.ylim(0.04, 0.1)
plt.plot(p1, label="processing segmented but not cropped")
plt.plot(p2, label="processing segmented and cropped")
plt.legend()
print(np.average(p1))
print(np.average(p2))
df = pd.DataFrame()
df['processing segmented but not cropped'] = p1
df['processing segmented and cropped'] = p2
df.describe()

In [ ]:
import pandas as pd
plt.ylim(0, 0.003)
plt.plot(timings, label="segmented but not cropped")
plt.plot(timings_2, label="segmented and cropped")
plt.legend()
print(np.average(timings))
print(np.average(timings_2))
df = pd.DataFrame()
df['segmented but not cropped'] = timings
df['segmented and cropped'] = timings_2
df.describe()

In [ ]:
from PIL import Image
img = Image.open('framenocrop.jpg')
print(img.size)
img

In [ ]:
from PIL import Image
img = Image.open('frameoccrop.jpg')
print(img.size)
img

In [ ]:
import os
os.getpid()